In [1]:
import numpy as np
import pandas as pd

In [2]:
# Assumes the raw file is in the same directory as the notebook
data = pd.read_csv("unhcr_popstats_export_asylum_seekers_all_data.csv",skiprows=3)

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.columns

Index(['Year', 'Country / territory of asylum/residence', 'Origin',
       'RSD procedure type / level', 'Tota pending start-year',
       'of which UNHCR-assisted', 'Applied during year',
       'statistics.filter.decisions_recognized',
       'statistics.filter.decisions_other', 'Rejected', 'Otherwise closed',
       'Total decisions', 'Total pending end-year',
       'of which UNHCR-assisted.1'],
      dtype='object')

In [4]:
# No data before 2000
print("Earliest year of data: {}".format(data['Year'].min()))

Earliest year of data: 2000


In [5]:
# Annoyingly, the data uses both NaN and * for missing values.  This convert * to NaN which are then dropped.
data['Applied during year'].loc[data['Applied during year']=='*']=np.NaN

# Number of missing values in "Applied during year"
num_missing_values = sum(data['Applied during year'].isna())
print("Number of missing values in 'Applied during year': {}".format(num_missing_values))
print("Fraction of missing values in 'Applied during year': {}".format(num_missing_values/data.shape[0]))

Number of missing values in 'Applied during year': 11875
Fraction of missing values in 'Applied during year': 0.07870493107104984


//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
# For now, just remove the missing value rows
filtered_data = data.dropna(subset=['Applied during year'])
filtered_data = filtered_data.astype({'Applied during year': 'int32'})

In [7]:
# Group the data by year and origin, sum the values of 'Applied during year'
grouped_data = filtered_data.groupby(['Year','Origin'])['Applied during year'].sum()

In [8]:
# Check that totals line up
print("Grouped sum: {}".format(grouped_data.sum()))
print("Raw sum: {}".format(filtered_data['Applied during year'].sum()))

Grouped sum: 22615322
Raw sum: 22615322


In [10]:
# Convert to a DataFrame so we can add columns
output_df = pd.DataFrame(grouped_data)
output_df['year_origin'] = output_df.index

In [12]:
# Create csv
output_df.to_csv("Asylum_seekers_per_country_per_year.csv")